# Final Group Assignment
- Analyzing news articles to determine if they are fake or real, based on a dataset from kaggle.

### 1. Analysis: Frame the problem and look at the big picture
2. Define the objective in business terms: Create a model capable of recognizing fake and real news.
3. How should you frame the problem (supervised/unsupervised etc.)?: Supervised learning, with binary labeling (1 for real, 0 for fake news) 
4. How should performance be measured?: Performance will be measured by the model accuracy, correctly classified vs. all.

In [ ]:
#imports
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from gensim.models import Word2Vec

### 2. Get the data
Data were retrieved from Kaggle (link below). This data set consists of 2 csv files, one with the real and other with the real news. Both data sets have 'Title', 'Text', 'Subject' and 'Date' columns. 

https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset

In the next step we labeled fake and real data, with 0 and 1 respectively and then merged them into 1 dataframe and shuffled them.

In [ ]:
#Load & label fake news data
df_fake = pd.read_csv('Fake.csv')
df_fake['label'] = 0

In [ ]:
#Load & label real news data
df_true = pd.read_csv('True.csv')
df_true['label'] = 1

#rename all 'politicsNews' to 'politics' in df_true
df_true['subject'] = df_true['subject'].replace('politicsNews', 'politics')

In [ ]:
#print distinct subjects
print(df_fake.subject.unique())
print(df_true.subject.unique())

In [ ]:
#merge & shuffle data
df_news = pd.concat([df_fake, df_true]).sample(frac=1).reset_index(drop=True)
df_news.head()

# Visualization

In [ ]:
#group data by subject and label
df_news.groupby(['subject', 'label']).size()

In [ ]:
#fake vs real news
df_news.groupby(['label']).size().plot(kind='bar', color=['red', 'blue']).set_xticklabels(['Fake', 'Real'], rotation=0)

In [ ]:
#number of articles per subject and label
df_news.groupby(['subject', 'label']).size().unstack().plot(kind='bar', stacked=False, figsize=(8, 5))

In [ ]:
#print the full text of the first 5 articles of subject 'worldnews'
for i in range(5):
    print(df_news[(df_news['subject'] == 'worldnews') & (df_news['label'] == 1)]['text'].iloc[i])
    print('---------------------------------')

In [ ]:
#average word count of articles per label
df_news.groupby(['label'])['text'].apply(lambda x: x.str.split().str.len().mean()).plot(kind='bar', color=['red', 'blue']).set_xticklabels(['Fake', 'Real'], rotation=0)

In [ ]:
#average word count of articles per subject per label
df_news.groupby(['subject', 'label'])['text'].apply(lambda x: x.str.split().str.len().mean()).unstack().plot(kind='bar', figsize=(8, 5))

# Correlation
correlation matrix: Title-subject-Label?

In [ ]:
#correlation between date and label
df_news.groupby(['date', 'label']).size().unstack().plot(kind='line', figsize=(15, 5))

In [ ]:
#add column 'contains_Reuters' that will be 1 if the article contains the word 'Reuters' and 0 otherwise
df_news['contains_Reuters'] = df_news['text'].apply(lambda x: 1 if 'Reuters' in x else 0)

#correlation between label and 'contains_Reuters'
df_news.groupby(['contains_Reuters', 'label']).size().unstack().plot(kind='bar', color=['red', 'blue']).set_xticklabels(['No', 'Yes'], rotation=0)

In [ ]:
# dropping date and subject
df_news = df_news.drop(columns = ['subject','date'])

In [ ]:
df_news.head(10)

# I. Splitting the data and vectorizing using TF-IDF (Term Frequency - Inverse Document Frequency)
- additionaly, we are testing it out with a Multinomial Naive Bayes classifier

## a) Splitting the data, while including the title in X

In [ ]:
X = df_news['title'] + ' ' + df_news['text']
y = df_news['label']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.6)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the validation and test data using the same vectorizer
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

## b) Using Grid Search to perform 5 fold cross-validation in order to get the best params and estimator for MultinomialNaiveBayes

In [ ]:
param_grid = {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]}
nb_classifier = MultinomialNB()
grid_search = GridSearchCV(estimator=nb_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train_tfidf, y_train)

best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

best_classifier = grid_search.best_estimator_

y_val_pred = best_classifier.predict(X_val_tfidf)

val_accuracy = accuracy_score(y_val, y_val_pred)
print(f'Validation Accuracy: {val_accuracy:.2f}')

y_test_pred = best_classifier.predict(X_test_tfidf)

test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Test Accuracy: {test_accuracy:.2f}')

print(classification_report(y_test, y_test_pred))

### Ultimately, it seems like MNB yielded impressively good results

## c) We do the same using a CNN to investigate whether it will output better results than the MNB Classifier

## d) Using the same vectorizer, we try it out with RNN to analyze if this will yield more accuracy overall

# II. Using WordEmbeddings for text representation (Word2Vec)

In [ ]:
X = df_news['title'] + ' ' + df_news['text']
y = df_news['label']

# Convert labels to numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
max_words = 10000  # Adjust based on the size of your vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure consistent length for input to neural network
max_sequence_length = 1000  # Adjust based on your dataset
X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_sequence_length)

# Train Word2Vec model
tokenized_sentences = [text.split() for text in X_train]
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Create an embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        try:
            embedding_vector = word2vec_model.wv[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            # Word not in Word2Vec model vocabulary
            pass

# Build a simple neural network model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model on the test set
y_pred = model.predict(X_test_padded)
y_pred = y_pred.flatten()

# Convert predictions back to original labels
y_pred_labels = label_encoder.inverse_transform(((y_pred > 0.5).astype(int)))
print(y_pred_labels)
print(y_test)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_labels)
print(f'Test Accuracy: {accuracy:.2f}')

# Print classification report for the test set
print(classification_report(y_test, y_pred_labels))